In [2]:
pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample data for user preferences
data = {
    'User': ['Alice', 'Alice', 'Alice', 'Bob', 'Bob', 'Bob', 'Charlie', 'Charlie', 'Charlie'],
    'Item': ['Notebook', 'Pen', 'Book', 'Notebook', 'Book', 'Eraser', 'Pen', 'Book', 'Eraser'],
    'Rating': [5, 4, 3, 4, 5, 2, 5, 4, 3]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Create a user-item matrix
user_item_matrix = df.pivot_table(index='User', columns='Item', values='Rating').fillna(0)

# Compute user similarity using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Recommendation function
def recommend_items(user, num_recommendations=3):
    # Get user's ratings
    user_ratings = user_item_matrix.loc[user]

    # Get similarity scores for the user
    similar_users = user_similarity_df[user].sort_values(ascending=False)

    # Weighted ratings for recommendation
    weighted_ratings = pd.Series()

    for similar_user, score in similar_users.items():
        if similar_user == user:
            continue  # Skip the current user
        # Add weighted ratings from similar users
        weighted_ratings = weighted_ratings.add(user_item_matrix.loc[similar_user] * score, fill_value=0)

    # Normalize the weighted ratings
    recommendations = weighted_ratings / similar_users.sum()

    # Exclude items already rated by the user
    recommendations = recommendations[user_ratings[user_ratings > 0].index]

    # Return the top recommended items
    return recommendations.sort_values(ascending=False).head(num_recommendations)

# Example usage
if __name__ == "__main__":
    user = 'Alice'
    print(f"Recommendations for {user}:")
    print(recommend_items(user))


Recommendations for Alice:
Item
Book        2.628124
Pen         1.345745
Notebook    1.241222
dtype: object
